# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [1]:
# Part 1
prob_matrix = [[0.3, 0.4, 0.3], [0.2, 0.5, 0.3], [0.4, 0.3, 0.3]]
start, end = 1, 0
prob1 = 0
for i in range(3):
    prob1 += prob_matrix[start][i] * prob_matrix[i][end]
# print(prob1)
# Fill in the answer to part 1 below as a decimal number
problem1_p1 = prob1

In [2]:
# Part 2
prob2 = 0
for i in range(1, 3):
    prob2 += prob_matrix[start][i] * prob_matrix[i][end]
# print(prob2)
# Fill in the answer to part 2 below as a decimal number
problem1_p2 = prob2

In [3]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [4]:
# Part 4
import numpy as np
transition_matrix = np.array(prob_matrix)
# print(transition_matrix)
steps = 10**3
n_transition_matrix = transition_matrix
for i in range(steps):
    n_transition_matrix = np.matmul(n_transition_matrix, transition_matrix)
# print(n_transition_matrix)
# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary = np.around(n_transition_matrix[0], 2)
print(problem1_stationary)

[0.29 0.41 0.3 ]


In [5]:
# Part 5
new_prob_matrix = [[1, 0, 0], [0.2, 0.5, 0.3], [0.4, 0.3, 0.3]]
A = np.matrix([[0.5, -0.3], [-0.3, 0.7]])
B = np.matrix([[1], [1]])
A_inverse = np.linalg.inv(A)
X = np.around(A_inverse * B, 1)
# print(X)
# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
problem1_ET = float(X[0])
print(problem1_ET)

3.8


---
## Assignment 2, PROBLEM 2
Maximum Points = 8


A healthcare organization is interested in understanding the relationship between the number of visits to the doctors office and certain patient characteristics. 
They have collected data on the number of visits (for a year) for a sample of patients and have included the following variables

* ofp : number of physician office visits
* ofnp : number of nonphysician office visits
* opp : number of physician outpatient visits
* opnp : number of nonphysician outpatient visits
* emr : number of emergency room visits
* hosp : number of hospitalizations
* exclhlth : the person is of excellent health (self-perceived)
* poorhealth : the person is of poor health (self-perceived)
* numchron : number of chronic conditions
* adldiff : the person has a condition that limits activities of daily living ?
* noreast : the person is from the north east region
* midwest : the person is from the midwest region
* west : the person is from the west region
* age : age in years (divided by 10)
* male : is the person male ?
* married : is the person married ?
* school : number of years of education
* faminc : family income in 10000$
* employed : is the person employed ?
* privins : is the person covered by private health insurance?
* medicaid : is the person covered by medicaid ?

Decide which patient features are resonable to use to predict the target "number of physician office visits". Hint: should we really use the "ofnp" etc variables?

Since the target variable is counts, a reasonable loss function is to consider the target variable as Poisson distributed where the parameter follows $\lambda = \exp(\alpha \cdot x + \beta)$ where $\alpha$ is a vector (slope) and $\beta$ is a number (intercept). That is, the parameter is the exponential of a linear function. The reason we chose this as our parameter, is that it is always positive which is when the Poisson distribution is defined. To be specific we make the following assumption about our conditional density of $Y \mid X$,
$$
    f_{Y \mid X} (y,x) = \frac{\lambda^{y} e^{-\lambda}}{y !}, \quad \lambda(x) = \exp(\alpha \cdot x + \beta).
$$

Recall from the lecture notes, (4.2) that in this case we should consider the log-loss (entropy) and that according to (4.2.1 Maximum Likelihood and regression) we can consider the conditional log-likelihood. Follow the steps of Example 1 and Example 2 in section (4.2) to derive the loss that needs to be minimized.

Hint: when taking the log of the conditional density you will find that the term that contains the $y!$ does not depend on $\lambda$ and as such does not depend on $\alpha,\beta$, it can thus be discarded. This will be essential due to numerical issues with factorials.

Instructions:

1. Load the file `data/visits_clean.csv`, follow the instructions in the code cell of how this should happen [1.5p]
2. Create the `problem2_X` and the `problem2_y` as numpy arrays with `problem2_X` being the features and `problem2_y` being the target. Do the standard train-test split with 80% training data and 20% testing data. Store these in the variables defined in the cells. [1.5p]
3. Implement $loss$ inside the class `PoissonRegression` by writing down the loss to be minimized, I have provided a formula for the $\lambda$ that you can use. [1.5p]
4. Now use the `PoissonRegression` class to train a Poisson regression model on the training data. [1.5p]
5. Compute the mean absolute error of your prediction on the test set and use Hoeffdings inequality to produce a 95\% confidence interval for the mean absolute error. We can make the assumption that the error is bounded by 70 for simplicity. [2p]


In [26]:
# Part 1
import csv

data = []
cleaned_data = []

with open('data/visits_clean.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)
    for val in csv_reader:
        data.append(val)
# print(''.join(header).split(' '))
# print(data)

for d in data:
    d_list = ''.join(d).split(' ')
    d_list = [float(dle) for dle in d_list]
    cleaned_data.append(d_list)
# print(cleaned_data)
# As in assignment 1 we will load the header into header and data into data
# this time you will have to parse the data such that each data entry is a float
# and that the problem2_data is a numpy array of shape (n_samples,n_columns)
# where n_columns is the number of columns and should have the same length as
# the list of strings header. n_samples is how many rows of data we had.
# If you cannot find the file, check the starting package as it should be updated
# if not, go to the github repo and pull it

# The autograder does not accept pandas as a solution to this problem.
# data/visits_clean.csv
problem2_header = ''.join(header).split(' ') # [""] #List of strings

n_samples, n_columns = len(data), len(problem2_header)
# print(str(n_samples) + ' ' + str(n_columns))

problem2_data = np.array(cleaned_data) #A numpy array of shape n_samples n_columns
# print(problem2_data.shape)
# print(problem2_data)

In [27]:
# Part 2

# Fill in your X and y below
problem2_X = problem2_data[:, 6:]
problem2_y = problem2_data[:, 0]
# print(problem2_X)
# print(problem2_y)

seed = np.random.randint(0, 10000)
np.random.seed(seed)
np.random.shuffle(problem2_X)
np.random.seed(seed)
np.random.shuffle(problem2_y)

import math
ratio = 0.8
split_index = math.ceil(0.8*n_samples)
# print(split_index)
# Split the data into train and randomly using for instance
# np.random.shuffle indices and indexing the first 80% as the train data
# keep the train size as 0.8 rounded up to the nearest integer sample
problem2_X_train, problem2_X_test, problem2_y_train, problem2_y_test = problem2_X[:split_index], problem2_X[split_index:], problem2_y[:split_index], problem2_y[split_index:]

In [28]:
# Part 3

# Fill in the function loss below

class PoissonRegression(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        # The parameter lambda for the given X and the proposed values 
        # of the coefficients, here coeff[:-1] represent alpha 
        # and coeff[-1] represent beta
        lam = np.exp(X @ coeffs[:-1] + coeffs[-1])

        # use the Y variable that is available here to define 
        # the loss function, return the value of the loss for 
        # this Y and for this parameter lam defined above
        return (lam - np.log(lam)*Y).mean()

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the loss above together with an optimization method from scipy
        #to find the coefficients of the model
        #this is prepared for you below
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1) # initial guess as 0
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            return np.exp(np.dot(X,self.coeffs[:-1])+self.coeffs[-1])

In [29]:
# Part 4

# Initialize your PoissonRegression model
problem2_model = PoissonRegression()

# Fit your initialized model on the training data
problem2_model.fit(problem2_X_train, problem2_y_train)

# This is to make sure that everything went well, 
# check that success is True
print(problem2_model.result)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -4.776501683453863
       x: [-4.656e-01  2.945e-01 ...  2.693e-01  1.018e+00]
     nit: 536
     jac: [ 5.960e-08 -4.768e-07 ... -1.132e-06 -8.047e-06]
    nfev: 19346
    njev: 1138


In [30]:
# Part 5
# print(len(problem2_y_test))
# print('-------------------------------------')
y_prediction = problem2_model.predict(problem2_X_test)
# print(len(y_prediction))
n = len(y_prediction)
# sample_mean = y_prediction.mean()
# print(sample_mean)
# true_mean = problem2_y_test.mean()
# print(true_mean)
err_sum = 0
for i in range(n):
    err_sum += abs(problem2_y_test[i] - y_prediction[i])
    
# Put the computed mean absolute error in the variable below
problem2_metric = err_sum/n
# print(problem2_metric)
# Put a confidence interval in the variable below by using Hoeffdings inequality using the bounds
# a = 0, b=70 (roughly 5 days between visits as minimum)
# the variable should contain a tuple representing the confidence interval of the form (l_edge,r_edge)
a = 0
b = 70
epsilon = math.sqrt(-((b-a)**2/(2*len(y_prediction)))*math.log(0.025))
# print(epsilon)
problem2_interval = (problem2_metric-epsilon, problem2_metric+epsilon)
# problem2_interval_1 = (problem2_metric-epsilon, problem2_metric+epsilon)
# print(problem2_interval)
# print(problem2_interval_1)

---
## Assignment 2, PROBLEM 3
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [31]:
from Utils import linConGen
def problem3_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    m,a,b = (2**32, 16777217,12349)
    lcg = []
    u = seed
    
    for s in range(size):
        u = (a*u + b) % m
        lcg.append(u)
    
    return lcg

In [32]:

def problem3_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    gen_list = generator(size, seed)
    uniform_pseudo_random = list(np.array(gen_list)/period)
    
    return uniform_pseudo_random

In [33]:

def problem3_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    samples = []
    rejections = 0
    while (len(samples) < n_iterations):
        x = uniformGenerator(1, seed)[0]
        y = (np.pi/2)*abs(np.sin(2*np.pi*x))
        z = uniformGenerator(1, seed)[0]
        if (z <= y):
            samples.append(x)
        else:
            rejections += 1
        seed += 1
    #print(samples, rejections)
    return samples

---
#### Local Test for Assignment 2, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [34]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem3_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem3_uniform(generator=problem3_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem3_uniform(generator=problem3_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [16789566, 1056989307, 3120599224, 1912652021, 1728114994, 2566988143, 134304172, 3019997673, 2634134054, 3271680611]
Uniform sampler [0.003909125458449125, 0.2460995006840676, 0.7265711259096861, 0.44532400113530457, 0.40235812636092305, 0.5976735015865415, 0.031270126812160015, 0.7031480020377785, 0.613307127263397, 0.7617475024890155]
Accept-Reject sampler [0.003909125458449125, 0.007815375691279769, 0.011721625924110413, 0.015627876156941056, 0.0195341263897717, 0.023440376622602344, 0.027346626855432987, 0.03125287708826363, 0.035159127321094275, 0.03906537755392492, 0.04297162778675556, 0.046877878019586205, 0.05078412825241685, 0.05469037848524749, 0.058596628718078136, 0.06250287895090878, 0.06640912918373942, 0.07031537941657007, 0.07422162964940071, 0.07812787988223135]


In [35]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.13436424411240122, 0.23796462709189137, 0.23604808973743452, 0.6229016948897019, 0.793340083761663, 0.32383276483316237, 0.2267058593810488, 0.5714025946899135, 0.4523795535098186, 0.2590084917154736, 0.10682853770165568, 0.36152277491407514, 0.18126486333322134, 0.6771258268002703, 0.16494947983319797, 0.7123429878269185, 0.376962302390386, 0.7472912662232836, 0.6484972199788831, 0.11295717017616302]
